<a href="https://colab.research.google.com/github/diablaxus/Red-Neuronal-Multicapa/blob/main/MulticapaPyTorch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Ejercicio 2: Red Neuronal Multicapa en PyTorch para un problema de clasificación

# Paso 1: Importar las bibliotecas necesarias
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Paso 2: Definir los hiperparámetros
input_size = 784  # 28x28 imágenes de MNIST aplanadas
hidden_size = 128  # Número de neuronas en la capa oculta
num_classes = 10  # MNIST tiene 10 clases (dígitos 0-9)
batch_size = 64  # Tamaño del lote
learning_rate = 0.001  # Tasa de aprendizaje para Adam
num_epochs = 10  # Número de épocas

# Paso 3: Descargar y cargar el conjunto de datos MNIST
# Definir transformaciones: convertir imágenes a tensores y normalizar valores de píxeles
transform = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))])  # Normalizar entre -1 y 1

# Cargar el conjunto de datos MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Crea un iterador para cargar datos en lotes
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Paso 4: Definir el modelo de red neuronal
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Capa densa 1: entrada -> 128 neuronas ocultas
        self.relu = nn.ReLU()  # Función de activación ReLU
        self.fc2 = nn.Linear(hidden_size, num_classes)  # Capa densa 2: 128 -> 10 salidas (dígitos)

    def forward(self, x):
        x = x.view(-1, input_size)  # Aplanar la imagen de 28x28 a 784
        x = self.fc1(x)  # Paso por la primera capa densa
        x = self.relu(x)  # Aplicar ReLU
        x = self.fc2(x)  # Paso por la segunda capa densa (capa de salida)
        return x

# Crear una instancia del modelo
model = NeuralNet(input_size, hidden_size, num_classes)

# Paso 5: Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()  # Entropía cruzada
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Optimizer Adam

# Paso 6: Entrenar el modelo
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Pasa las imágenes a través del modelo
        outputs = model(images)

        # Calcula la pérdida
        loss = criterion(outputs, labels)

        # Limpia los gradientes acumulados
        optimizer.zero_grad()

        # Calcula los gradientes de la pérdida
        loss.backward()

        # Actualiza los parámetros del modelo
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Paso 7: Evaluar el modelo
model.eval()  # Cambiar el modelo al modo de evaluación
with torch.no_grad():  # Desactivar el cálculo de gradientes
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Obtener la clase con mayor probabilidad
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Precisión en el conjunto de prueba: {100 * correct / total:.2f}%')


Epoch [1/10], Loss: 0.2364
Epoch [2/10], Loss: 0.0780
Epoch [3/10], Loss: 0.2126
Epoch [4/10], Loss: 0.0223
Epoch [5/10], Loss: 0.0256
Epoch [6/10], Loss: 0.0164
Epoch [7/10], Loss: 0.0443
Epoch [8/10], Loss: 0.0062
Epoch [9/10], Loss: 0.0247
Epoch [10/10], Loss: 0.0025
Precisión en el conjunto de prueba: 97.13%
